# Solar model - version 2

Date of creation: 17.06.2020

Last updated: 4.07.2020

In [1]:
import math
import numpy as np
import pandas as pd

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
import physics as ph
from Sun import SSM18 as s

Physics version 1.4 9.07.2020
SSM18 version 1.7 4.07.2020


## Функции для модели

## Интегрирование от центра по расстоянию
параметры модели: плотность и температура в центре.

хим.состав, число точек, шаг по радиусу.

In [17]:
def model_R(d0, t0, Z=0.02, dim=30000, step=s.R/10000, M=s.M, debug=False) :
    dr = step  
    # distance from the center
    r = np.zeros(dim, dtype='float64')
    # density
    d = np.zeros(dim, dtype='float64')
    # temperature
    t = np.zeros(dim, dtype='float64')
    # светимость
    l = np.zeros(dim, dtype='float64')
    # mass inside r
    m = np.zeros(dim, dtype='float64')
    # pressure
    p = np.zeros(dim, dtype='float64')
    # hydrogen
    x = np.zeros(dim, dtype='float64')
    # opacity
    k = np.zeros(dim, dtype='float64')
    # 1 - convection, 0 - radiation
    c = np.zeros(dim)
    # скорость выгорания водорода (еличина, обратная характерному времени)
    v = np.zeros(dim, dtype='float64')
    
# начальные данные
    for n in range(2) :
        r[n] = n*step
        V = 4*ph.pi/3*pow(r[n], 3)
        d[n] = d0
        t[n] = t0
        x[n] = .732
        X = x[0]
        Y = 1 - Z - x[0]
        m[n] = V*d[n]
        l[n] = m[n]*ph.Etot(d[n], t[n], X, Y, Z)
        p[n] = ph.Pressure(d[n], t[n], X, Y, Z)
        k[n] = ph.Opacity(d[n], t[n], X, Y, Z)
        v[n] = 4*ph.m_prot*ph.Etot(d[n], t[n], X, Y, Z)/X/ph.dEpp*ph.year
        mu = ph.MU(X, Y, Z)
        S = 4*ph.pi*r[n]*r[n]
        g = ph.G*m[n]/r[n]/r[n]
        rad = 3/16*k[n]*d[n]*l[n]/ph.sigma/pow(t[n], 3)/S
        con = 0.4*mu/ph.kB*ph.m_prot*g
        if rad < con:
            c[n] = 0.
        else:
            c[n] = 1.
    for n in range(1, dim - 1) :
        x[n] = .732
        X = x[0]
        Y = 1 - Z - x[0]
        r[n+1] = r[n] + dr 
        mu = ph.MU(X, Y, Z)
        S = 4*ph.pi*r[n]*r[n]
        g = ph.G*m[n]/r[n]/r[n]
        k[n] = ph.Opacity(d[n], t[n], X, Y, Z)
        rad = -3/16*k[n]*d[n]*l[n]/ph.sigma/pow(t[n], 3)/S
        con = -0.4*mu/ph.kB*ph.m_prot*g
        if abs(rad) < abs(con):
            c[n] = 0.
            dtdr = rad
        else:
            c[n] = 1.
            dtdr = con
        dt = 2*dtdr*dr - (t[n] - t[n-1])/(r[n] - r[n-1])*dr
        t[n+1] = t[n] + dt
        dpdr = -g*d[n]
        p[n+1] = p[n] + 2*dpdr*dr - (p[n] -p[n-1])/(r[n] - r[n-1])*dr
        # Находим плотность из уравнения состояния
        d[n+1] = mu*ph.m_prot*p[n+1]/ph.kB/t[n+1]
        davg = (d[n+1] + d[n])/2
        tavg = (t[n+1] + t[n])/2
        dm = 4*ph.pi/3*(pow(r[n+1], 3) - pow(r[n], 3))*davg
        m[n+1] = m[n] + dm
        l[n+1] = l[n] + dm*ph.Etot(davg, tavg, X, Y, Z)
        v[n] = 4*ph.m_prot*ph.Etot(d[n], t[n], X, Y, Z)/X/ph.dEpp*ph.year
        if debug:
            print("%d r=%.4f m=%.4f d=%.3f t=%.0f" % (n, r[n]/s.R, m[n]/M, d[n], t[n]))
        # уменьшаем шаг если температура падает слишком резко
        if abs(dt/t[n]) > 1e-3 :
            dr = 0.7*dr
        if m[n]/M > 1.2 : # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            raise Exception('Mass exceeded 1.2*M_target')
        Tsur = pow(l[n]/ph.sigma/S, 1/4)
        if t[n] < Tsur :
            break
        if p[n] < 0. : 
            raise Exception("Negative pressure")
            break

    if (n == dim - 2) :
         raise Exception("Error, increase dim!")

    df = pd.DataFrame({'Mass': m[:n], 'Radius': r[:n], 'Temperature': t[:n], 'Density': d[:n], 'Luminosity': l[:n], 
                       'Hydrogen': x[:n], 'Opacity': k[:n], 'Convection': c[:n], 'Pressure': p[:n], 'Velocity': v[:n]})

    return (n, m[n], r[n], l[n], t[n], df)

In [19]:
%%time
T0 = (13.9)*1e6
den0 = 100
n, M, R, L, T, df = model_R(den0, T0, debug=True)


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in double_scalars
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars


1 r=0.0001 m=0.0000 d=100.000 t=13900000
2 r=0.0002 m=0.0000 d=100.000 t=13899993
3 r=0.0003 m=0.0000 d=100.000 t=13899986
4 r=0.0004 m=0.0000 d=100.000 t=13899971
5 r=0.0005 m=0.0000 d=99.999 t=13899957
6 r=0.0006 m=0.0000 d=99.999 t=13899935
7 r=0.0007 m=0.0000 d=99.999 t=13899913
8 r=0.0008 m=0.0000 d=99.999 t=13899884
9 r=0.0009 m=0.0000 d=99.998 t=13899855
10 r=0.0010 m=0.0000 d=99.998 t=13899819
11 r=0.0011 m=0.0000 d=99.997 t=13899783
12 r=0.0012 m=0.0000 d=99.997 t=13899739
13 r=0.0013 m=0.0000 d=99.996 t=13899696
14 r=0.0014 m=0.0000 d=99.996 t=13899645
15 r=0.0015 m=0.0000 d=99.995 t=13899594
16 r=0.0016 m=0.0000 d=99.994 t=13899536
17 r=0.0017 m=0.0000 d=99.994 t=13899478
18 r=0.0018 m=0.0000 d=99.993 t=13899413
19 r=0.0019 m=0.0000 d=99.992 t=13899348
20 r=0.0020 m=0.0000 d=99.991 t=13899275
21 r=0.0021 m=0.0000 d=99.990 t=13899203
22 r=0.0022 m=0.0000 d=99.989 t=13899123
23 r=0.0023 m=0.0000 d=99.988 t=13899043
24 r=0.0024 m=0.0000 d=99.987 t=13898956
25 r=0.0025 m=0.0000 

767 r=0.0767 m=0.0296 d=87.658 t=12873750
768 r=0.0768 m=0.0297 d=87.628 t=12871185
769 r=0.0769 m=0.0298 d=87.598 t=12868621
770 r=0.0770 m=0.0299 d=87.568 t=12866050
771 r=0.0771 m=0.0300 d=87.537 t=12863481
772 r=0.0772 m=0.0302 d=87.507 t=12860906
773 r=0.0773 m=0.0303 d=87.477 t=12858331
774 r=0.0774 m=0.0304 d=87.446 t=12855750
775 r=0.0775 m=0.0305 d=87.416 t=12853171
776 r=0.0776 m=0.0306 d=87.385 t=12850585
777 r=0.0777 m=0.0307 d=87.355 t=12848000
778 r=0.0778 m=0.0308 d=87.325 t=12845409
779 r=0.0779 m=0.0309 d=87.294 t=12842819
780 r=0.0780 m=0.0311 d=87.263 t=12840223
781 r=0.0781 m=0.0312 d=87.233 t=12837628
782 r=0.0782 m=0.0313 d=87.202 t=12835027
783 r=0.0783 m=0.0314 d=87.172 t=12832426
784 r=0.0784 m=0.0315 d=87.141 t=12829820
785 r=0.0785 m=0.0316 d=87.110 t=12827215
786 r=0.0786 m=0.0317 d=87.080 t=12824604
787 r=0.0787 m=0.0319 d=87.049 t=12821993
788 r=0.0788 m=0.0320 d=87.018 t=12819377
789 r=0.0789 m=0.0321 d=86.987 t=12816761
790 r=0.0790 m=0.0322 d=86.956 t=1

1228 r=0.1228 m=0.1077 d=71.463 t=11405884
1229 r=0.1229 m=0.1080 d=71.426 t=11401997
1230 r=0.1230 m=0.1082 d=71.388 t=11398104
1231 r=0.1231 m=0.1084 d=71.350 t=11394209
1232 r=0.1232 m=0.1087 d=71.312 t=11390306
1233 r=0.1233 m=0.1089 d=71.274 t=11386402
1234 r=0.1234 m=0.1091 d=71.236 t=11382490
1235 r=0.1235 m=0.1093 d=71.199 t=11378577
1236 r=0.1236 m=0.1096 d=71.161 t=11374655
1237 r=0.1237 m=0.1098 d=71.123 t=11370733
1238 r=0.1238 m=0.1100 d=71.085 t=11366802
1239 r=0.1239 m=0.1103 d=71.047 t=11362870
1240 r=0.1240 m=0.1105 d=71.010 t=11358930
1241 r=0.1241 m=0.1107 d=70.972 t=11354989
1242 r=0.1242 m=0.1110 d=70.934 t=11351039
1243 r=0.1243 m=0.1112 d=70.896 t=11347088
1244 r=0.1244 m=0.1114 d=70.859 t=11343129
1245 r=0.1245 m=0.1117 d=70.821 t=11339170
1246 r=0.1246 m=0.1119 d=70.783 t=11335201
1247 r=0.1247 m=0.1121 d=70.745 t=11331232
1248 r=0.1248 m=0.1124 d=70.708 t=11327254
1249 r=0.1249 m=0.1126 d=70.670 t=11323276
1250 r=0.1250 m=0.1128 d=70.632 t=11319289
1251 r=0.12

1747 r=0.1747 m=0.2565 d=51.215 t=9136043
1748 r=0.1748 m=0.2569 d=51.176 t=9131370
1749 r=0.1749 m=0.2572 d=51.137 t=9126700
1750 r=0.1750 m=0.2575 d=51.097 t=9122025
1751 r=0.1751 m=0.2579 d=51.058 t=9117354
1752 r=0.1752 m=0.2582 d=51.019 t=9112677
1753 r=0.1753 m=0.2585 d=50.980 t=9108005
1754 r=0.1754 m=0.2589 d=50.940 t=9103327
1755 r=0.1755 m=0.2592 d=50.901 t=9098653
1756 r=0.1756 m=0.2595 d=50.862 t=9093973
1757 r=0.1757 m=0.2599 d=50.823 t=9089298
1758 r=0.1758 m=0.2602 d=50.783 t=9084617
1759 r=0.1759 m=0.2605 d=50.744 t=9079941
1760 r=0.1760 m=0.2609 d=50.705 t=9075259
1761 r=0.1761 m=0.2612 d=50.666 t=9070581
1762 r=0.1762 m=0.2615 d=50.626 t=9065897
1763 r=0.1763 m=0.2619 d=50.587 t=9061218
1764 r=0.1764 m=0.2622 d=50.548 t=9056533
1765 r=0.1765 m=0.2625 d=50.509 t=9051852
1766 r=0.1766 m=0.2629 d=50.470 t=9047166
1767 r=0.1767 m=0.2632 d=50.431 t=9042484
1768 r=0.1768 m=0.2636 d=50.391 t=9037797
1769 r=0.1769 m=0.2639 d=50.352 t=9033113
1770 r=0.1770 m=0.2642 d=50.313 t=

...

27807 r=0.3877 m=0.7544 d=0.009 t=403379
27808 r=0.3877 m=0.7544 d=0.008 t=402977
27809 r=0.3877 m=0.7544 d=0.009 t=403379
27810 r=0.3877 m=0.7544 d=0.008 t=402977
27811 r=0.3877 m=0.7544 d=0.009 t=403379
27812 r=0.3877 m=0.7544 d=0.008 t=402977
27813 r=0.3877 m=0.7544 d=0.009 t=403379
27814 r=0.3877 m=0.7544 d=0.008 t=402976
27815 r=0.3877 m=0.7544 d=0.009 t=403379
27816 r=0.3877 m=0.7544 d=0.008 t=402976
27817 r=0.3877 m=0.7544 d=0.009 t=403378
27818 r=0.3877 m=0.7544 d=0.008 t=402976
27819 r=0.3877 m=0.7544 d=0.009 t=403378
27820 r=0.3877 m=0.7544 d=0.008 t=402976
27821 r=0.3877 m=0.7544 d=0.009 t=403378
27822 r=0.3877 m=0.7544 d=0.008 t=402976
27823 r=0.3877 m=0.7544 d=0.009 t=403378
27824 r=0.3877 m=0.7544 d=0.008 t=402976
27825 r=0.3877 m=0.7544 d=0.009 t=403378
27826 r=0.3877 m=0.7544 d=0.008 t=402976
27827 r=0.3877 m=0.7544 d=0.009 t=403378
27828 r=0.3877 m=0.7544 d=0.008 t=402975
27829 r=0.3877 m=0.7544 d=0.009 t=403378
27830 r=0.3877 m=0.7544 d=0.008 t=402975
27831 r=0.3877 m

28528 r=0.3878 m=0.7544 d=0.008 t=403057
28529 r=0.3878 m=0.7544 d=0.009 t=403343
28530 r=0.3878 m=0.7544 d=0.008 t=403057
28531 r=0.3878 m=0.7544 d=0.009 t=403343
28532 r=0.3878 m=0.7544 d=0.008 t=403057
28533 r=0.3878 m=0.7544 d=0.009 t=403342
28534 r=0.3878 m=0.7544 d=0.008 t=403057
28535 r=0.3878 m=0.7544 d=0.009 t=403342
28536 r=0.3878 m=0.7544 d=0.008 t=403057
28537 r=0.3878 m=0.7544 d=0.009 t=403342
28538 r=0.3878 m=0.7544 d=0.008 t=403057
28539 r=0.3878 m=0.7544 d=0.009 t=403342
28540 r=0.3878 m=0.7544 d=0.008 t=403057
28541 r=0.3878 m=0.7544 d=0.009 t=403342
28542 r=0.3878 m=0.7544 d=0.008 t=403056
28543 r=0.3878 m=0.7544 d=0.009 t=403342
28544 r=0.3878 m=0.7544 d=0.008 t=403056
28545 r=0.3878 m=0.7544 d=0.009 t=403342
28546 r=0.3878 m=0.7544 d=0.008 t=403056
28547 r=0.3878 m=0.7544 d=0.009 t=403342
28548 r=0.3878 m=0.7544 d=0.008 t=403056
28549 r=0.3878 m=0.7544 d=0.009 t=403342
28550 r=0.3878 m=0.7544 d=0.008 t=403056
28551 r=0.3878 m=0.7544 d=0.009 t=403342
28552 r=0.3878 m

29239 r=0.3878 m=0.7544 d=0.009 t=403308
29240 r=0.3878 m=0.7544 d=0.008 t=403019
29241 r=0.3878 m=0.7544 d=0.009 t=403308
29242 r=0.3878 m=0.7544 d=0.008 t=403019
29243 r=0.3878 m=0.7544 d=0.009 t=403308
29244 r=0.3878 m=0.7544 d=0.008 t=403019
29245 r=0.3878 m=0.7544 d=0.009 t=403308
29246 r=0.3878 m=0.7544 d=0.008 t=403019
29247 r=0.3878 m=0.7544 d=0.009 t=403308
29248 r=0.3878 m=0.7544 d=0.008 t=403019
29249 r=0.3878 m=0.7544 d=0.009 t=403308
29250 r=0.3878 m=0.7544 d=0.008 t=403019
29251 r=0.3878 m=0.7544 d=0.009 t=403308
29252 r=0.3878 m=0.7544 d=0.008 t=403018
29253 r=0.3878 m=0.7544 d=0.009 t=403308
29254 r=0.3878 m=0.7544 d=0.008 t=403018
29255 r=0.3878 m=0.7544 d=0.009 t=403308
29256 r=0.3878 m=0.7544 d=0.008 t=403018
29257 r=0.3878 m=0.7544 d=0.009 t=403308
29258 r=0.3878 m=0.7544 d=0.008 t=403018
29259 r=0.3878 m=0.7544 d=0.009 t=403307
29260 r=0.3878 m=0.7544 d=0.008 t=403018
29261 r=0.3878 m=0.7544 d=0.009 t=403307
29262 r=0.3878 m=0.7544 d=0.008 t=403018
29263 r=0.3878 m

29918 r=0.3878 m=0.7544 d=0.008 t=402983
29919 r=0.3878 m=0.7544 d=0.009 t=403276
29920 r=0.3878 m=0.7544 d=0.008 t=402983
29921 r=0.3878 m=0.7544 d=0.009 t=403276
29922 r=0.3878 m=0.7544 d=0.008 t=402983
29923 r=0.3878 m=0.7544 d=0.009 t=403275
29924 r=0.3878 m=0.7544 d=0.008 t=402983
29925 r=0.3878 m=0.7544 d=0.009 t=403275
29926 r=0.3878 m=0.7544 d=0.008 t=402982
29927 r=0.3878 m=0.7544 d=0.009 t=403275
29928 r=0.3878 m=0.7544 d=0.008 t=402982
29929 r=0.3878 m=0.7544 d=0.009 t=403275
29930 r=0.3878 m=0.7544 d=0.008 t=402982
29931 r=0.3878 m=0.7544 d=0.009 t=403275
29932 r=0.3878 m=0.7544 d=0.008 t=402982
29933 r=0.3878 m=0.7544 d=0.009 t=403275
29934 r=0.3878 m=0.7544 d=0.008 t=402982
29935 r=0.3878 m=0.7544 d=0.009 t=403275
29936 r=0.3878 m=0.7544 d=0.008 t=402982
29937 r=0.3878 m=0.7544 d=0.009 t=403275
29938 r=0.3878 m=0.7544 d=0.008 t=402982
29939 r=0.3878 m=0.7544 d=0.009 t=403275
29940 r=0.3878 m=0.7544 d=0.008 t=402982
29941 r=0.3878 m=0.7544 d=0.009 t=403275
29942 r=0.3878 m

Exception: Error, increase dim!

In [62]:
print('T0=%.5f D0=%.10f n=%d M=%.4f R=%.4f L=%.4f T=%.0f' % (T0, den0, n, M/M_sol, R/R_sol, L/L_sol, T))

T0=13900000.00000 D0=90.0000000000 n=10419 M=0.8090 R=0.4289 L=0.9254 T=8648


In [14]:
T0 = (13.31)*1e6
den0 = 95
n, M, R, L, T, df = model_R(den0, T0, debug=False)
n, M/M_sol, R/R_sol, L/L_sol, T

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in double_scalars
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars


Exception: Mass exceeded 1.2*M_target

### Поиск комбинации центральных плотности и температуры, дающих решение с массой равной солнечной. (Первый вариант)

Для интегрирования уравнений из центра надо задать плотность и температуру в центре, но только одна их комбинации 
соответствует решению с массой, равной солнечной (при фиксированном хим. составе)

Внешний цикл по температурам, внутренний цикл - по плотности.
При фиксированной температуре постепенно повышаем плотность, пока не появится стабильное решение с массой < 1.
После этого возвращаемся на 1 шаг назад по плотности, уменьшаем шаг по плотности в 10 раз и повторям процедуру.
Так делаем пока шаг по плотности не станет меньше 1e-8.

In [90]:
%%time
dim = 1 # 11
for j in range(dim) :
    # T0 = 1.333e7 + 0.001e7*j  # M = M_sol, epoch = 0
    T0 = 13.9e6 + 0.1e6*j
    step = 10 # step by density
    exc = True
    den0 = 30
    if j > 0:
        df_prev = df.copy()
    while step > 1e-8 :    
        for i in range(10) :
            try :
                n, M, R, L, T, df = model_R(den0, T0, debug=False)
            except Exception as error:
#                 print(T0, den0, 'Caught this error: ' + repr(error))
                exc = True
                den0 += step
                break
            if exc :
                exc = False
                print('\tT0=%.5e D0=%.10f n=%d M=%.4f R=%.4f L=%.4f T=%.0f' % (T0, den0, n, M/M_sol, R/R_sol, L/L_sol, T))
                den0 -= step
                step /= 10.
                break
            
    print('# T0=%.5e D0=%.2f n=%d M=%.4f R=%.4f L=%.3f T=%.0f' % (T0, den0, n, M/M_sol, R/R_sol, L/L_sol, T))
    if M/M_sol > 1 :
#         df = df_prev
        break
print("Done!")

	T0=1.39000e+07 D0=220.0000000000 n=9182 M=0.4972 R=0.2670 L=1.0253 T=11248
	T0=1.39000e+07 D0=212.0000000000 n=9410 M=0.5712 R=0.3006 L=1.1707 T=10960
	T0=1.39000e+07 D0=211.4000000000 n=10050 M=0.6669 R=0.4021 L=1.2177 T=9566
	T0=1.39000e+07 D0=211.4000000000 n=10050 M=0.6669 R=0.4021 L=1.2177 T=9566
	T0=1.39000e+07 D0=211.3950000000 n=10476 M=0.6919 R=0.4773 L=1.2185 T=8785
	T0=1.39000e+07 D0=211.3947000000 n=10948 M=0.7034 R=0.5562 L=1.2186 T=8135
	T0=1.39000e+07 D0=211.3946900000 n=11128 M=0.7056 R=0.5853 L=1.2186 T=7932
	T0=1.39000e+07 D0=211.3946870000 n=11301 M=0.7070 R=0.6107 L=1.2186 T=7762
	T0=1.39000e+07 D0=211.3946866000 n=11353 M=0.7073 R=0.6183 L=1.2186 T=7718
	T0=1.39000e+07 D0=211.3946865700 n=11358 M=0.7074 R=0.6190 L=1.2186 T=7709
# T0=1.39000e+07 D0=211.39 n=11358 M=0.7074 R=0.6190 L=1.219 T=7709
Done!
Wall time: 2min 8s


In [49]:
df = df_prev
df

,Mass,Radius,Temperature,Density,Luminosity,Hydrogen,Opacity,Convection,Pressure,Velocity
0,0.000000e+00,0.000000e+00,1.333000e+07,94.554471,0.000000e+00,0.732,1.318613,0.0,1.730205e+17,8.561869e-11
1,1.335360e+26,6.960000e+07,1.333000e+07,94.554471,1.634396e+27,0.732,1.318613,1.0,1.730205e+17,8.561869e-11
2,1.735967e+26,7.656000e+07,1.332997e+07,94.554040,2.124711e+27,0.732,1.318620,0.0,1.730193e+17,8.561741e-11
3,2.220700e+26,8.352000e+07,1.332993e+07,94.553578,2.717979e+27,0.732,1.318627,0.0,1.730180e+17,8.561603e-11
4,2.797568e+26,9.048000e+07,1.332990e+07,94.553080,3.424001e+27,0.732,1.318634,0.0,1.730166e+17,8.561455e-11
...,...,...,...,...,...,...,...,...,...,...
13995,1.989737e+33,6.262796e+10,5.465099e+03,0.000031,2.461129e+33,0.732,0.003113,0.0,2.349260e+07,4.899800e-49
13996,1.989737e+33,6.262800e+10,5.460450e+03,0.000031,2.461129e+33,0.732,0.003109,0.0,2.344258e+07,4.849587e-49
13997,1.989737e+33,6.262805e+10,5.455801e+03,0.000031,2.461129e+33,0.732,0.003105,0.0,2.339263e+07,4.799847e-49
13998,1.989737e+33,6.262810e+10,5.451153e+03,0.000031,2.461129e+33,0.732,0.003101,0.0,2.334274e+07,4.750576e-49


In [50]:
_ = df.to_csv(line_terminator='\n')
filename = "FromCenter/1epoch%.02d.csv" % epoch
print(filename)
csv_file = open(filename, "wt")
n = csv_file.write(_)
csv_file.close()

FromCenter/1epoch00.csv


### Поиск комбинации центральных плотности и температуры, дающих решение с массой равной солнечной. (Второй вариант)

Для интегрирования уравнений из центра надо задать плотность и температуру в центре, но только одна их комбинации соответствует решению с массой, равной солнечной (при фиксированном хим. составе)

Альтернативный поиск (в обратном порядке)

Внешний цикл по плотности, внутренний цикл - по температуре. При фиксированной плотности постепенно понижаем температуру, пока не появится стабильное решение с массой < 1. После этого возвращаемся на 1 шаг назад по температуре, уменьшаем шаг по температуре в 10 раз и повторям процедуру. Так делаем пока шаг по температуре не станет меньше 0.01 градуса.

In [ ]:
%%time
# Another order of fit - density outside
dim = 11
for j in range(dim) :
    den0 = 94 + 0.1*j # 94.4 + 0.1*j
    step = 1e6 # step for Temperature
    exc = True
    T0 = 15
    if j > 0:
        df_prev = df.copy()
    while step > 1e-2 :    
        for i in range(10) :
            try :
                n, M, R, L, T, df = model_R(den0, T0, debug=False)
            except Exception as error:
#                 print(T0, den0, 'Caught this error: ' + repr(error))
                exc = True
                T0 -= step
                break
            if exc :
                exc = False
                print('\tT0=%.5f D0=%.10f n=%d M=%.4f R=%.4f L=%.4f T=%.0f' % (T0, den0, n, M/M_sol, R/R_sol, L/L_sol, T))
                T0 += step
                step /= 10.
                break
            
    print('# T0=%.5f D0=%.2f n=%d M=%.4f R=%.4f L=%.3f T=%.0f' % (T0, den0, n, M/M_sol, R/R_sol, L/L_sol, T))
    if M/M_sol > 1. :
#         df = df_prev
        break
print("Done!")

In [ ]:
# df = df_prev
df

In [ ]:
_ = df.to_csv(line_terminator='\n')
filename = "FromCenter/2epoch%.02d.csv" % epoch
print(filename)
csv_file = open(filename, "wt")
n = csv_file.write(_)
csv_file.close()

In [30]:
T0=1.33000e+07 
den0=94.4000000000
n, M, R, L, T, df = model_R(den0, T0, debug=False)
n, M/M_sol, R/R_sol, L/L_sol, T

(11049,
 0.8772552888605576,
 0.5009900889800694,
 0.6252592215835585,
 7252.863270233664)